In [1]:
import pandas as pd
import numpy as np
import datetime
import tweepy

def get_api(cfg):
    auth = tweepy.OAuthHandler(cfg['consumer_key'], cfg['consumer_secret'])
    auth.set_access_token(cfg['access_token'], cfg['access_token_secret'])
    return tweepy.API(auth)

cfg = { "consumer_key"        : "",
    "consumer_secret"     : "",
    "access_token"        : "-kpcqQiI6yPxIe1tRpuu9YnvAyhSja8l",
    "access_token_secret" : "" }
api = get_api(cfg)

In [2]:
tmp = pd.read_json('media_founders_dump.json')

In [3]:
tmp = tmp[~tmp.inn.isnull()]
tmp['inn'] = tmp['inn'].astype(int).astype(str)
tmp.loc[tmp['inn'].str.len()==9,'inn'] = tmp.loc[tmp['inn'].str.len()==9,'inn'].apply( lambda x: x.zfill(10))
tmp.loc[tmp['inn'].str.len()==11,'inn'] = tmp.loc[tmp['inn'].str.len()==11,'inn'].apply( lambda x: x.zfill(12))

In [4]:
tmp.shape

(30952, 6)

## Suppliers

In [5]:
supp_df = pd.read_json('suppliers-20171201.json')

In [6]:
# supp_df = supp_df[~supp_df.inn.isnull()]
supp_df['inn'] = supp_df['inn'].astype(str)
supp_df.loc[supp_df['inn'].str.len()==9,'inn'] = supp_df.loc[supp_df['inn'].str.len()==9,'inn'].apply( lambda x: x.zfill(10))
supp_df.loc[supp_df['inn'].str.len()==11,'inn'] = supp_df.loc[supp_df['inn'].str.len()==11,'inn'].apply( lambda x: x.zfill(12))

In [7]:
supp_df = supp_df[supp_df.inn.isin(tmp.inn)]

In [10]:
supp_df = supp_df[['inn','ogrn','kpp','organizationName','contracts223Sum','contracts223YearStats']]

In [11]:
supp_df.loc[~supp_df['contracts223YearStats'].isnull(),'2014_contracts'] = \
supp_df.loc[~supp_df['contracts223YearStats'].isnull(),'contracts223YearStats'].apply(lambda x: x.get('2014',0))

supp_df.loc[~supp_df['contracts223YearStats'].isnull(),'2015_contracts'] = \
supp_df.loc[~supp_df['contracts223YearStats'].isnull(),'contracts223YearStats'].apply(lambda x: x.get('2015',0))

supp_df.loc[~supp_df['contracts223YearStats'].isnull(),'2016_contracts'] = \
supp_df.loc[~supp_df['contracts223YearStats'].isnull(),'contracts223YearStats'].apply(lambda x: x.get('2016',0))

supp_df.loc[~supp_df['contracts223YearStats'].isnull(),'2017_contracts'] = \
supp_df.loc[~supp_df['contracts223YearStats'].isnull(),'contracts223YearStats'].apply(lambda x: x.get('2017',0))

In [68]:
supp_df = supp_df[~supp_df['contracts223YearStats'].isnull()]

In [12]:
def get_summ(dct):
    if type(dct)==dict:
        return dct.get('contractsSum')
    else:
        return 0

In [13]:
supp_df['2014_contracts'] = supp_df['2014_contracts'].apply(get_summ)
supp_df['2015_contracts'] = supp_df['2015_contracts'].apply(get_summ)
supp_df['2016_contracts'] = supp_df['2016_contracts'].apply(get_summ)
supp_df['2017_contracts'] = supp_df['2017_contracts'].apply(get_summ)

## Check clearspending

In [125]:
def check_contracts():
    try:
        dict_cont = pd.read_json("http://openapi.clearspending.ru/restapi/v3/contracts/search/?daterange=01.11.2017-19.12.2017")['contracts']['data']
        tmp = pd.DataFrame(dict_cont)
        tmp.publishDate = pd.to_datetime(tmp.publishDate)
        tmp = tmp[tmp.publishDate>=(datetime.datetime.now() - datetime.timedelta(hours=4))]
        tmp = tmp[['contractUrl', 'currency', 'number', 'price', 'publishDate', 'suppliers', 'products'  ]]
        tmp['in_owners'] = tmp['suppliers'].apply(lambda z: len(list(set([x.get('inn','') for x in z]) & set(supp_df.inn)))>0)
        tmp = tmp[tmp.in_owners]
        # отобрал собственников
        tmp['okdp_codes'] = tmp['products'].apply(lambda z:[x.get('OKPD2').get('code').split('.')[0] for x in z])
        tmp['smi_prod'] = tmp['okdp_codes'].apply(lambda z: len(list(set(z) & set(['58','59','60','61','62','63'])))>0)
        tmp['names']= tmp['suppliers'].apply(lambda z:[x.get('organizationName', 'None') for x in z])
        # пометил сми
        tmp.apply(lambda x:send_tweet(x[['publishDate','number','price','names','smi_prod']]), axis=1)
    except:
        api.update_status(status='No actual contracts available at {}'.format(datetime.datetime.now()))
        print('update error')
        pass


In [121]:
def send_tweet(df):
    df.columns=['Дата публикации', 'Номер', 'Цена', 'Участник', 'СМИ ОКПД']
    try:
        status = api.update_status(status=str(df)[:280])
    except:
        print ('twitter error')
    

In [126]:
import time
if __name__ == "__main__":
    for i in range(0,100):
        check_contracts()
        time.sleep(14400)
        print ('run again')

update error


KeyboardInterrupt: 